In [ ]:
# Import 

import os
from matplotlib import pyplot as plt
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

% matplotlib inline

In [ ]:
## Plant Color Density
# Color histogram code source: https://bit.ly/2ENtVXr

# code v1

image_small = cv2.imread("imgs/20190525_153508.jpg")
image_medium = cv2.imread("imgs/20190525_153521.jpg")
image_big = cv2.imread("imgs/20190525_153515.jpg")

is_ = cv2.cvtColor(image_small, cv2.COLOR_BGR2RGB) # cv2.COLOR_BGR2RGB cv2.COLOR_BGR2GRAY
im = cv2.cvtColor(image_medium, cv2.COLOR_BGR2RGB)
ib = cv2.cvtColor(image_big, cv2.COLOR_BGR2RGB)

w = 10
h = 10
fig = plt.figure(figsize=(8, 8))
columns = 3
rows = 1

for i in range(0, columns*rows):
    img = [is_, im, ib][i]
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img)
    
plt.show()

colors_small = color_hist(image_small)
green_area(colors_small)

colors_medium = color_hist(image_medium)
green_area(colors_medium)

colors_large = color_hist(image_big)
green_area(colors_large)

# code v2

def img_chist(path):
 
    # load the image and show it
    image = cv2.imread(path)
    # convert the image to grayscale and create a histogram
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    plt.imshow(image)
    plt.title('Pic')
    plt.show()

#     hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
#     plt.figure()
#     plt.title("Grayscale Histogram")
#     plt.xlabel("Bins")
#     plt.ylabel("# of Pixels")
#     plt.plot(hist)
#     plt.xlim([0, 256])

    # grab the image channels, initialize the tuple of colors,
    # the figure and the flattened feature vector
    chans = cv2.split(image)
    colors = ("b", "g", "r")
    plt.figure()
    plt.title("'Flattened' Color Histogram")
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")
    features = []

    # loop over the image channels
    for (chan, color) in zip(chans, colors):
        
        if color == "g":
            # create a histogram for the current channel and
            # concatenate the resulting histograms for each
            # channel
            hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
            features.extend(hist)

            # plot the histogram
            plt.plot(hist, color = color)
            plt.xlim([0, 256])

    # here we are simply showing the dimensionality of the
    # flattened color histogram 256 bins for each channel
    # x 3 channels = 768 total values -- in practice, we would
    # normally not use 256 bins for each channel, a choice
    # between 32-96 bins are normally used, but this tends
    # to be application dependent
    
    return("max green: ", np.max(hist))

# images in order of increasing size

p1 = "C:/Users/Gordon/Desktop/me/genspace/images/20190525_153508.jpg"
p2 = "C:/Users/Gordon/Desktop/me/genspace/images/20190525_153521.jpg"
p3 = "C:/Users/Gordon/Desktop/me/genspace/images/20190525_153515.jpg"
p4 = "C:/Users/Gordon/Desktop/me/genspace/images/20190525_153527.jpg"

In [ ]:
# Plant Size

# Assumed 90mm instead of 60 mm for diameter of petri dish
#! python object_size.py --image images/20190525_153521.jpg --width 3.54331

#im = cv2.imread("./pics/DSC_4223.jpg")
#imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray,127,255,0)
#image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#img = cv2.drawContours(im, contours, -1, (0,255,0), 1)

#cv2.imshow('contour',img)
#cv2.waitKey(0)
#cv2.imwrite('contour.png',img)
 
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

img = "./pics/DSC_4223.jpg"
width = 3.54331 # 2.3622

# load the image, convert it to grayscale, and blur it slightly
image = cv2.imread(img)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)
 
# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations = 1)
edged = cv2.erode(edged, None, iterations = 1)
 
# find contours in the edge map
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
 
# sort the contours from left-to-right and initialize the
# 'pixels per metric' calibration variable
(cnts, _) = contours.sort_contours(cnts)
pixelsPerMetric = None

# loop over the contours individually
for c in cnts:
    # if the contour is not sufficiently large, ignore it
    if cv2.contourArea(c) < 100:
        continue

    # compute the rotated bounding box of the contour
    orig = image.copy()
    box = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    box = np.array(box, dtype="int")

    # order the points in the contour such that they appear
    # in top-left, top-right, bottom-right, and bottom-left
    # order, then draw the outline of the rotated bounding
    # box
    box = perspective.order_points(box)
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

    # loop over the original points and draw them
    for (x, y) in box:
        cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
        
    # unpack the ordered bounding box, then compute the midpoint
    # between the top-left and top-right coordinates, followed by
    # the midpoint between bottom-left and bottom-right coordinates
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)

    # compute the midpoint between the top-left and top-right points,
    # followed by the midpoint between the top-righ and bottom-right
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)

    # draw the midpoints on the image
    cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

    # draw lines between the midpoints
    cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
        (255, 0, 255), 2)
    cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
        (255, 0, 255), 2)
    
    # compute the Euclidean distance between the midpoints
    dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
    dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

    # if the pixels per metric has not been initialized, then
    # compute it as the ratio of pixels to supplied metric
    # (in this case, inches)
    if pixelsPerMetric is None:
        pixelsPerMetric = dB / width

            # compute the size of the object
    dimA = dA / pixelsPerMetric
    dimB = dB / pixelsPerMetric

    # draw the object sizes on the image
    cv2.putText(orig, "{:.1f}in".format(dimA),
        (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
        0.65, (255, 255, 255), 2)
    cv2.putText(orig, "{:.1f}in".format(dimB),
        (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
        0.65, (255, 255, 255), 2)

    # show the output image
    #plt.imshow(orig)
    cv2.imshow("Image", orig)
    cv2.waitKey(0)